# More spatial functions

## Measurement

distance(geometry) - called on a geometry and returns distance to another geometry.


In [ ]:
%matplotlib inline
import geopandas as gpd

eagle = gpd.read_file("data/BAEA_Nests.shp")
eagle.to_crs(epsg=26913, inplace=True)
eagle.plot()

In [ ]:
eagle['geometry'].y.mean()

In [ ]:
eagle['geometry'].x.mean()

In [ ]:
import shapely.geometry as shp
eagle_center = shp.Point(eagle['geometry'].x.mean(), eagle['geometry'].y.mean())
eagle['distance_to_center'] = eagle['geometry'].distance(eagle_center)
eagle.plot(column='distance_to_center', cmap='Reds')

In [ ]:
eagle.sort_values('distance_to_center', ascending=False).head(16)

In [ ]:
eagle = eagle[eagle['distance_to_center']<46000]

## Spatial predicates

Compare two geometries and return True or False

We discussed the intersects predicate previously but there are many more.

*.intersects(other)* returns true if any part of the geometry it is called on touches any part of another geometry

*.equals(other)* returns true if the geometries are exactly equivalent

*.almost_equal(other, tolerance)* returns true if the geometries are equal within the given tolerance level.

*.contains(other)* returns true if the geometry it is called in completely encompasses the other

*.within(other)* the opposite of contains

Other spatial predicates include *touches, crosses, overlap, disjoint*

You can read the [shapely documentation](https://shapely.readthedocs.io/en/stable/manual.html) for more information about these predicates.

More complex relstionships can be formed using the DE91M method.  More information on this can be foud in [wikipedia](https://en.wikipedia.org/wiki/DE-9IM#:~:text=compact%20representation%20of%20.-,Spatial%20predicates,defined%20for%20some%20common%20relations.) or in the shapely documentation

Lets use the within predicate to view raptor nests within Boulder county.  First we'll get a polygon representing Boulder County

In [ ]:
counties = gpd.read_file("data/colorado_counties.shp")
boulder_county = counties[counties['NAMELSAD10']=='Boulder County'].unary_union
boulder_county

Next we'll read in the raptor file to a GeoDataFrame

In [ ]:
raptors = gpd.read_file("data/Raptor_Nests.shp")
raptors

Now lets view the subset of raptor nests within Boulder county

In [ ]:
boulder_nests = raptors[raptors['geometry'].within(boulder_county)]
boulder_nests.describe()

We can see that this dataset contains only 40 nests out of the original 879

In [ ]:
basemap = counties[counties['NAMELSAD10']=='Boulder County'].plot(facecolor='none', edgecolor='k', figsize=(15, 15))
boulder_nests.plot(ax=basemap, color='red')

And when we plot it out we can see that we do only see the nests tht are WITHIN Boulder County.

The thing to remember is that spatial predicates test a relationshipo between two geometries and return True or False.  They can be used anywhere a logical expression is called for such as when subsetting a GeoDataFrame.

# Spatial operators - return geometry

We have alread looked at the intersection operator.  An intersection is basically a spatial AND operator. It returns the area that is in both geometries.

Others include union which is a spatial OR operator and includes the area that is in one geometry OR the other.

Symetric difference is a spatial XOR operator.  It includes areas that are in one OR the other but NOT in both.

Difference is a spatial subtraction. It removes the area in the other geometry from the geometry that it is called on. Notice that unlike the other 3 spatial oerators, the order in which they are evaluated is important.  In other words $(A - B) \ne (B - A)$. Again, more info is available in the [Shapely users manual](https://shapely.readthedocs.io/en/stable/manual.html).

Lets look at an example of using the difference operator to make ring buffers around the burrowing owl habitat.  First we will read in the buowl habitat shape file, then convert it to UTM coordinates and create a 300m buffer

In [ ]:
buowl = gpd.read_file("data/BUOWL_Habitat.shp")
buowl.to_crs(epsg=26913, inplace=True)
buowl['buffer'] = buowl['geometry'].buffer(300)

Next lets create ring buffers by subtracting the original geometry from the buffer.  We also need to fix some invalid geometries in the original data by calling a buffer of size 0 which will often recreate invalid geometries in proper form

In [ ]:
buowl['geometry'] = buowl['geometry'].buffer(0) # Necessary to fix invalid geometry
buowl['ring_buffer'] = buowl['buffer'].difference(buowl['geometry'])
buowl.set_geometry('ring_buffer').cx[500000:520000, 4440000:4460000].plot(figsize=(15,15))

If we calculate the areas of all three geometries we see that the area of the original geometry plus the ring buffer = the area of the buffer

In [ ]:
buowl['area_ha']=buowl['geometry'].area/10000
buowl['buffer_ha']=buowl['buffer'].area/10000
buowl['ring_ha']=buowl['ring_buffer'].area/10000
buowl['buf_minus_ring_ha']=buowl['buffer_ha']-buowl['ring_ha']
buowl

## The overlay method

Takes two complete data frames as a parameter plus a how parameter.

We have loooked at the case where how='intersection' which returns a new data frame in much the same way as performing an intersection in desktop GIS.

Other possibilities include union, identity, difference, and symetric_difference

Lets perform a difference operation that will subtract 1 mile buffers from raptor nests from Weld county and return it as a dataframe called 'unimpacted_weld'

In [ ]:
counties.to_crs(epsg=26913, inplace=True)
raptors.to_crs(epsg=26913, inplace=True) #Convert both dataframes to UTM coordinates

raptors['buffer']= raptors['geometry'].buffer(1600)
raptors.set_geometry('buffer', inplace=True) #Create 1 mile buffers around raptor nests and set to default geometry

unimpacted_weld = gpd.overlay(counties[counties['NAMELSAD10']=='Weld County'], raptors, how='difference')

unimpacted_weld.plot(figsize=(15,15))